In [33]:
import pandas as pd
import numpy as np
import math

In [2]:
data_dir = '/Users/annagreene/code/district-comparison/data/'

In [56]:
mapping = pd.read_csv(data_dir + 'raw/final_mapping.csv')
raw_dist = pd.read_csv(data_dir + 'raw/arizonaExample.csv')
test = pd.read_csv(data_dir + 'raw/atlantaExample.csv')
pd.options.mode.chained_assignment = None

In [109]:
raw_dist["Object_code"] = 0
raw_dist["Object_code"] = raw_dist.Account.str.split(".").str[3]
dist_1 = raw_dist[raw_dist['Account Type']=="EXPENDITURE"]
dist_1 = dist_1.drop(columns=['Active', 'Account', 'Description', 'Account Type', 'Budget Control Group', 'Budget', 'Budget Balance','Balance', 'Encumbrance', 'Pre Encumbrance', 'Pending Invoices', 'Uncommitted Balance'])
replace = [',', '$', '-', '(', ')']
dist_1['YTD Transactions'] = dist_1['YTD Transactions'].str.translate({ord(x): '' for x in replace}).astype(float)
dist_1 = dist_1.groupby(['Object_code'])['YTD Transactions'].sum().reset_index()
dist_1 = dist_1.rename(columns={'YTD Transactions': 'Transactions', 'Object_code': 'Code'})
dist_1 = dist_1.round(2)
dist_1['Code'] = dist_1['Code'].astype(float).astype(int)
dist_1

,Code,Transactions
0,6000,0.00
1,6100,0.00
2,6101,0.00
3,6103,0.00
4,6110,8002.11
...,...,...
181,6890,455508.18
182,6891,0.00
183,6892,0.00
184,6893,0.00


In [110]:
test = test[~test.DESCRIPTION.str.contains('TOTAL')]
test = test.drop(columns=['Unnamed: 1', 'DESCRIPTION'])
test = test.rename(columns={'ACCOUNT': 'Object_code', ' AMOUNT ': 'Transactions'})
replace = [',', '$', '-', '(', ')']
test['Transactions'] = test['Transactions'].str.translate({ord(x): '' for x in replace})
test['Transactions'] = np.double(test['Transactions'])
test = test.groupby(['Object_code'])['Transactions'].sum().reset_index()
test = test.round(2)
test['Object_code'] = test['Object_code'].str[5:8]
test

AttributeError: 'DataFrame' object has no attribute 'DESCRIPTION'

In [111]:
cols = ["Object_code","Function_code", "Category_Edstruments","Major_Category_Edstruments","Title", "State"]
clean_map = mapping[cols]
clean_map = clean_map.fillna(0)
clean_map["Code"] = ''
clean_map["Code"] = np.where(clean_map["Object_code"] == 0, clean_map["Function_code"], clean_map["Object_code"])
clean_map["Code"] = clean_map["Code"].astype(str)
clean_map = clean_map[clean_map["Code"].str.contains('X')==False]
clean_map["Code"] = clean_map["Code"].astype(float).astype(int)
clean_map

,Object_code,Function_code,Category_Edstruments,Major_Category_Edstruments,Title,State,Code
0,10,0,Teachers Salaries,Salaries,Regular Teacher,Alabama,10
1,11,0,Teachers Salaries,Salaries,Resource Teacher,Alabama,11
2,12,0,Teachers Salaries,Salaries,Alternative School Teacher,Alabama,12
3,14,0,Teachers Salaries,Salaries,Teacher - Gifted,Alabama,14
4,15,0,Special Education Salaries,Salaries,Teacher - Collaborative Special Education,Alabama,15
...,...,...,...,...,...,...,...
7962,728,0,Miscellaneous,Other Financing Uses,Transfers to Agency Funds,Wyoming,728
7963,729,0,Miscellaneous,Other Financing Uses,Transfers to Any Other Funds,Wyoming,729
7964,730,0,Miscellaneous,Other Financing Uses,Payments to Escrow Agents,Wyoming,730
7965,740,0,Debt Service,Other Financing Uses,Redemption of Principal Other,Wyoming,740


In [120]:
dist_1_map = clean_map[clean_map['State']=="Arizona"].reset_index()
test_merge3 = dist_1_map.merge(dist_1, how='outer', indicator=True)
test_merge4 = test_merge3[test_merge3['_merge'] == 'right_only']
not_mapped = list(test_merge4['Code'])
dist_1_map

,index,Object_code,Function_code,Category_Edstruments,Major_Category_Edstruments,Title,State,Code
0,2637,6100,0,Other Salaries,Salaries,PERSONAL SERVICES - SALARIES,Arizona,6100
1,2638,6110,0,Other Salaries,Salaries,Certified Salaries,Arizona,6110
2,2639,6111,0,Administrative Salaries,Salaries,Administrators,Arizona,6111
3,2640,6112,0,Teachers Salaries,Salaries,Teachers,Arizona,6112
4,2641,6113,0,Teachers Salaries,Salaries,Substitute Teachers,Arizona,6113
...,...,...,...,...,...,...,...,...
129,2766,6910,0,Miscellaneous,Other Financing Uses,Indirect Costs Transfers-Out,Arizona,6910
130,2767,6930,0,Miscellaneous,Other Financing Uses,Fund Transfers-Out,Arizona,6930
131,2768,6940,0,Miscellaneous,Other Financing Uses,Payment to Escrow Agent for Defeasance of Debt,Arizona,6940
132,2769,6950,0,Miscellaneous,Other Financing Uses,Special Items,Arizona,6950


In [81]:
dist_1_map = clean_map[clean_map['State']=="Arizona"].reset_index()

dist_1_merge_diff = dist_1.merge(dist_1_map, on='Code', how = 'outer', indicator = True).loc[lambda x : x['_merge']=='left_only']
dist_1_diff = list(dist_1_merge_diff['Code'])
dist_1['Code'] = dist_1['Code'].apply(lambda x: math.floor(x/10)*10 if x in dist_1_diff else x)

dist_1 = dist_1.groupby('Code')['Transactions'].sum().reset_index()
dist_1 = dist_1[dist_1['Transactions']>0]
dist_1

,Code,Transactions
2,6110,8002.11
3,6111,4490251.59
4,6112,35105711.03
5,6113,398413.43
6,6114,4904906.23
...,...,...
112,6832,1156779.29
114,6841,5195243.06
115,6842,313302.52
117,6860,26612.14


In [97]:
test_merge1 = dist_1.merge(dist_1_map, on='Code', how = 'outer', indicator = True).loc[lambda x : x['_merge']=='left_only']
test_merge2 = dist_1.merge(dist_1_map)
test_merge3 = dist_1.merge(dist_1_map, how='outer', indicator=True)
test_merge4 = test_merge3[test_merge3['_merge'] == 'left_only']
test_merge4

,Code,Transactions,index,Object_code,Function_code,Category_Edstruments,Major_Category_Edstruments,Title,State,_merge
